In [1]:
from __future__ import print_function, division
import os
import torch
import numpy as np
from torch.autograd import Variable
import librosa
from scipy.stats import mode
import csv
import scipy
from keras.preprocessing.sequence import pad_sequences
from model import LSTMnet
import random
from tqdm import tqdm
from csv import writer
import json

Using TensorFlow backend.
/home/a/miniconda3/envs/winston_base/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/a/miniconda3/envs/winston_base/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/a/miniconda3/envs/winston_base/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/a/minico

In [2]:
# Ignore warnings & Fix random seed
import warnings
warnings.filterwarnings("ignore")
random.seed(999)

In [3]:
def Spec_and_Phase(fpath):
    signal, rate  = librosa.load(fpath, sr=16000)
    signal = signal/np.max(abs(signal)) # Restrict value between [-1,1]
    F = librosa.stft(signal, n_fft=512, hop_length=256, win_length=512, window=scipy.signal.hamming)
    spec = np.abs(F)
    phase = np.angle(F)
    spec = np.reshape(spec.T,(spec.shape[1],spec.shape[0]))
    return spec

# Split Original batch Data into Small-Chunk batch Data Structure with padding
def SmallChunkSplitData(data, FrameSize):  
    start = 0
    Start = [0]
    End = []
    Split_Data = []
    if len(data)>=FrameSize:
        equeal_division_data = data[:(int(len(data)/FrameSize))*FrameSize]
        split_data = np.split(equeal_division_data,int(len(data)/FrameSize))           
        left_data = data[len(equeal_division_data):]
        if len(left_data)!=0:            
            pad_left_data = pad_sequences(left_data.T, maxlen=FrameSize ,dtype='float', padding='post', truncating='post')
            pad_left_data = pad_left_data.T                         
            Split_Data = Split_Data + split_data + [pad_left_data]
            Start.append(start+len(split_data)+1)
            End.append(start+len(split_data)+1)
            start = start+len(split_data)+1
        else:
            Split_Data = Split_Data + split_data
            Start.append(start+len(split_data))
            End.append(start+len(split_data))
            start = start+len(split_data)  
    else:
        left_data = data
        pad_left_data = pad_sequences(left_data.T, maxlen=FrameSize ,dtype='float', padding='post', truncating='post')
        pad_left_data = pad_left_data.T
        Split_Data = Split_Data + [pad_left_data]        
        Start.append(start+1)
        End.append(start+1)
        start = start+1 
    return np.array(Split_Data)

def prediction_folder(input_path):
    F_Name = []
    Pred_Rsl = []
    for root, directories, files in os.walk(input_path):
        # print(root)
        # files = sorted(files)
        # print(files)
        print('Gender Predictions')
        for filename in tqdm(files):
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            if '.wav' in filepath:
                try:
                    data = Spec_and_Phase(filepath)
                    chunk_data = SmallChunkSplitData(data, FrameSize=65)
                    # Data to torch & float for model input
                    chunk_data = torch.from_numpy(chunk_data)
                    chunk_data = chunk_data.float().to(device)
                    # Pred-chunk-labels for chunk data
                    pred_label = model(chunk_data)
                    pred_label = (np.round( (Variable(pred_label).data).cpu().numpy() )).reshape(-1)
                    # Output Results
                    F_Name.append(filename)
                    Pred_Rsl.append(mode(pred_label)[0][0])   # output voting result only 
                except:
                    print('Cannot Predict: '+filename)
    return F_Name, Pred_Rsl

In [4]:
def to_csv(F_Name, Pred_Rsl):
    # f = open('/home/podcast/Desktop/DUMMY_ENV/INPUTS_OUTPUTS/Outputs/pred_gender_result_v1.csv','w')
    with open('/home/podcast/Desktop/MSP_Podcast_FILTER/emotions_feats_and_preds/pred_gender_result.csv','a', newline='') as f_object:
        # w = csv.writer(f)
        # w.writerow(('File_Name','Pred_Gender'))
        writer_object = writer(f_object)

        for i in range(len(F_Name)):
            gender = Pred_Rsl[i]
            if gender==1:
                gender = 'Male'
            elif gender==0:
                gender = 'Female'
            writer_object.writerow([F_Name[i], gender])
        f_object.close()
        print('Gender Done')
###############################################################################

In [5]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")      


In [6]:
# Settings  
Training_epochs = 15
feat_type = 'Spec'
MODEL_STRUCT = 'LSTM'
LOADING_PATH = './'+feat_type+'_'+MODEL_STRUCT+'_epoch'+str(Training_epochs)+'.pt.tar'

In [7]:
# Loading Model Parameters
model = LSTMnet(input_dim=257, hidden_dim=150, output_dim=1, num_layers=2)
model.load_state_dict(torch.load(LOADING_PATH, map_location=torch.device(device)))
model = model.to(device) # if wants to predict on torch.tensor data
model.eval()

LSTMnet(
  (lstm): Sequential(
    (lstm): LSTM(257, 150, num_layers=2, batch_first=True, dropout=0.5)
  )
  (dense): Sequential(
    (dense1): Linear(in_features=150, out_features=100, bias=True)
    (relu1): LeakyReLU(negative_slope=0.01)
    (dense2): Linear(in_features=100, out_features=50, bias=True)
    (relu2): LeakyReLU(negative_slope=0.01)
    (dense_out): Linear(in_features=50, out_features=1, bias=True)
    (act_out): Sigmoid()
  )
)

In [8]:
root = '/home/a/Desktop/MSP-Podcast/pipeline/'

output_location = 'INPUTS_OUTPUTS/Outputs/'

with open(os.path.join(os.path.join(root, output_location), "Short_files.json"), "r") as openfile:
    audio_files = json.load(openfile)

In [9]:
for filename, f_info in tqdm(audio_files.items()):
    f_path = f_info['filepaths']['wav']
    # music_info = f_info['gender']
    data = Spec_and_Phase(f_path)
    chunk_data = SmallChunkSplitData(data, FrameSize=65)
    # Data to torch & float for model input
    chunk_data = torch.from_numpy(chunk_data)
    chunk_data = chunk_data.float().to(device)
    # Pred-chunk-labels for chunk data
    pred_label = model(chunk_data)
    pred_label = (np.round( (Variable(pred_label).data).cpu().numpy() )).reshape(-1)
    # Output Results
    gender_pred = float(mode(pred_label)[0][0])   # output voting result only 
    if gender_pred == 1:
        gender_pred = 'Male'
    elif gender_pred == 0:
        gender_pred = 'Female'
    f_info['gender'] = gender_pred

100%|██████████| 499/499 [00:06<00:00, 81.25it/s]


In [10]:
json_object = json.dumps(audio_files, indent=4)
with open(os.path.join(root, output_location, "Short_files.json"), "w") as outfile:
    outfile.write(json_object)